# Sandbox

For playing around and testing. The contents aren't meant to makes sense afterwards or even run in order.

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch

In [72]:
qwe = torch.normal(torch.zeros(42, 128), 1)
qwe = qwe + qwe.min(keepdim=True, dim=-1)[0].abs()
asd = torch.nn.functional.normalize(qwe, p=1, dim=1)
asd[12].sum()

tensor(1.0000)

In [63]:
torch.normal(torch.zeros(42, 128), 1).shape

torch.Size([42, 128])

In [81]:
a = torch.tensor([[1, 2, 3], [2, 3, 4]])
b = a.pow(2).sum(keepdim=True, dim=-1).sqrt()

(a / b).pow(2).sum(dim=-1)

tensor([1.0000, 1.0000])